In [8]:
import numpy as np
import pandas as pd
data = np.array(pd.read_csv('bsl2.csv',header=None))


In [9]:
x = data[:,1:]
y = data[:,0]


In [10]:
x.shape

(757, 420)

In [11]:
def spxy(x, y, test_size=0.2):

    x_backup = x
    y_backup = y   
    M = x.shape[0]            
    N = round((1-test_size) * M)
    samples = np.arange(M)    
 
    y = (y - np.mean(y))/np.std(y)
    D = np.zeros((M, M))
    Dy = np.zeros((M, M))
 
    for i in range(M-1):
        xa = x[i, :]
        ya = y[i]
        for j in range((i+1), M):
            xb = x[j, :]
            yb = y[j]
            D[i, j] = np.linalg.norm(xa-xb)   
            Dy[i, j] = np.linalg.norm(ya - yb) 
 
    Dmax = np.max(D)       
    Dymax = np.max(Dy)
    D = D/Dmax + Dy/Dymax 
 
    maxD = D.max(axis=0)              
    index_row = D.argmax(axis=0)     #返回axis轴方向最大值的索引
    index_column = maxD.argmax()      
 
    m = np.zeros(N)
    m[0] = index_row[index_column]
    m[1] = index_column
    m = m.astype(int)        
 
    dminmax = np.zeros(N)
    dminmax[1] = D[m[0], m[1]]  
 
    for i in range(2, N): 
        pool = np.delete(samples, m[:i])
        dmin = np.zeros(M-i)  
        for j in range(M-i): 
            indexa = pool[j] 
            d = np.zeros(i)  
            for k in range(i):  
                indexb = m[k] 
                if indexa < indexb:  
                    d[k] = D[indexa, indexb]
                else:
                    d[k] = D[indexb, indexa]
            dmin[j] = np.min(d)    
        dminmax[i] = np.max(dmin)   
        index = np.argmax(dmin)    
        m[i] = pool[index]        
 
    m_complement = np.delete(np.arange(x.shape[0]), m)     
 
    spec_train = x[m, :]
    target_train = y_backup[m]
    spec_test = x[m_complement, :]
    target_test = y_backup[m_complement]
 
    return spec_train, spec_test, target_train, target_test

In [12]:
spec_train, spec_test, target_train, target_test = spxy(x,y, test_size=0.2)

In [13]:
train_data = np.c_[target_train,spec_train]
test_data = np.c_[target_test,spec_test]

In [14]:
np.savetxt('train.csv',train_data, delimiter=",", comments="", fmt='%s')
np.savetxt('test.csv',test_data, delimiter=",", comments="", fmt='%s')